# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic
- On-Policy
- 

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [ ] Experience Replay for Random Sampling
- [x] Importance Sampling
    
### Policy Gradient
- [x] MA-A3C

### Stability and Reducing Variance
- [x] Gradient clipping
- [ ] Normalized Reward/Advantage
- [x] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Synchronous Training (A2C)
- [x] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb
- https://arxiv.org/pdf/1706.02275.pdf

## TODO:

- Try to add experience buffer

In [1]:
!rm -rf logs/B4R4_nonzero_Population3/ model/B4R4_nonzero_Population3

In [2]:
TRAIN_NAME='B4R4_nonzero_Population3'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.25 # gpu capacity in percentage

In [3]:
import os

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random
import math

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import Experience_buffer, discount_rewards

from network.population import Population as Network

%load_ext autoreload
%autoreload 2

## Hyperparameters

In [4]:
# Replay Variables
total_episodes= 300000
max_ep = 150
update_frequency = 3
entropy_beta = 0.01
batch_size = 500
replay_capacity = 2000

# Saving Related
save_network_frequency = 1200
save_stat_frequency = 128
moving_average_step = 128

# Training Variables
decay_lr = False
lr_a = 5e-5
lr_c = 4e-4
lr_a_gamma = 0.9995
lr_c_gamma = 0.9995
lr_a_final = 1e-5
lr_c_final = 1e-4
lr_a_decay_step = (int) (math.log(lr_a_final/lr_a) / math.log(lr_a_gamma))
lr_c_decay_step = (int) (math.log(lr_c_final/lr_c) / math.log(lr_c_gamma))

gamma = 0.98 # discount_factor

# Env Settings
MAP_SIZE = 10
VISION_RANGE = 9 # What decide the network size !!!
VISION_dX, VISION_dY = 2*VISION_RANGE+1, 2*VISION_RANGE+1
in_size = [None,VISION_dX,VISION_dY,6]
num_policy_pool = 2

# Asynch Settings
global_scope = 'global'

## Environment Setting

In [ ]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [ ]:
action_space = 5
n_agent = 4

## A3C Network Structure

<img src="https://cdn-images-1.medium.com/max/1600/1*YtnGhtSAMnnHSL8PvS7t_w.png" width="450">

- Network is given in network.ActorCritic

## Environments

<img src="https://cdn-images-1.medium.com/max/1600/1*Hzql_1t0-wwDxiz0C97AcQ.png" width="450">
<img src="https://lilianweng.github.io/lil-log/assets/images/MADDPG.png" width="450">

In [ ]:
global_rewards = MA(moving_average_step)
global_ep_rewards = MA(moving_average_step)
global_length = MA(moving_average_step)
global_succeed = MA(moving_average_step)
global_episodes = 0

# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.Session()
progbar = tf.keras.utils.Progbar(total_episodes,interval=1)

## Worker

In [ ]:
class Worker(object):
    def __init__(self, name, global_network, sess, global_step=0):
        # Initialize Environment worker
        self.env = gym.make("cap-v0").unwrapped
        self.name = name
        
        # Create AC Network for Worker
        self.Network = Network(in_size=in_size,
                     action_size=action_space,
                     num_agent=n_agent,
                     decay_lr=decay_lr,
                     lr_actor=lr_a,
                     lr_critic=lr_c,
                     grad_clip_norm=0,
                     scope=self.name,
                     global_step=None,
                     initial_step=0,
                     entropy_beta = 0.01,
                     sess=sess,
                     global_network=global_network,
                     num_policy_pool=num_policy_pool,
                     allow_policy_share=True,
                     asynch_training = False)
        
        self.sess=sess
        
    def work(self, saver, writer):
        global global_rewards, global_ep_rewards, global_episodes, global_length, global_succeed, MAP_SIZE
        total_step = 1
        local_ep = 0
        buffer = Experience_buffer(experience_shape=6,
                                   buffer_size=replay_capacity)
                
        with self.sess.as_default(), self.sess.graph.as_default():
            while global_episodes < total_episodes:
                local_ep += 1
                s0 = self.env.reset(map_size=MAP_SIZE, policy_red=policy.zeros.PolicyGen(self.env.get_map, self.env.get_team_red))
                #s0 = one_hot_encoder(s0, self.env.get_team_blue, VISION_RANGE)
                s0 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                a1, a_probs1, v1 = self.Network.get_ac(s0)
                
                # parameters 
                ep_r = 0 # Episodic Reward
                prev_r = 0
                was_alive = [ag.isAlive for ag in self.env.get_team_blue]

                indv_history = [ [] for _ in self.env.get_team_blue ]
                
                for step in range(max_ep+1):
                    a, a_probs, v0 = a1, a_probs1, v1
                    
                    s1, rc, d, _ = self.env.step(a)
                    #s1 = one_hot_encoder(s1, self.env.get_team_blue, VISION_RANGE)
                    s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                    is_alive = [ag.isAlive for ag in self.env.get_team_blue]

                    r = (rc - prev_r-1)
                    if step == max_ep and d == False:
                        r = -100
                        rc = -100
                        d = True

                    r /= 100.0
                    ep_r += r

                    if d:
                        v1 = [0.0 for _ in range(len(self.env.get_team_blue))]
                    else:
                        a1, a_probs1, v1 = self.Network.get_ac(s1)

                    # push to buffer
                    for idx, agent in enumerate(self.env.get_team_blue):
                        if was_alive[idx]:
                            indv_history[idx].append([s0[idx],
                                                      a[idx],
                                                      r,
                                                      v0[idx],
                                                      a_probs[idx][a[idx]],
                                                      np.concatenate((s0[:idx],s0[idx+1:]),axis=0),
                                                      [1]+(is_alive[:idx]+is_alive[idx+1:])*1
                                                    ])

                    # Iteration
                    prev_r = rc
                    was_alive = is_alive
                    s0=s1
                    total_step += 1

                    if d:
                        alosses, closs = self.train(indv_history, sess, v1)
                        break
                        
                global_ep_rewards.append(ep_r)
                global_rewards.append(rc)
                global_length.append(step)
                global_succeed.append(self.env.blue_win)
                global_episodes += 1
                self.sess.run(global_step_next)
                progbar.update(global_episodes)
                if global_episodes % save_stat_frequency == 0 and global_episodes != 0:
                    summary = tf.Summary()
                    summary.value.add(tag='Records/mean_reward', simple_value=global_rewards())
                    summary.value.add(tag='Records/mean_length', simple_value=global_length())
                    summary.value.add(tag='Records/mean_succeed', simple_value=global_succeed())
                    summary.value.add(tag='Records/mean_episode_reward', simple_value=global_ep_rewards())
                    '''summary.value.add(tag='summary/Entropy', simple_value=etrpy)'''
                    for idx in range(n_agent):
                        summary.value.add(tag='summary/actor_loss_'+str(idx), simple_value=alosses[idx])
                    summary.value.add(tag='summary/critic_loss', simple_value=closs)
                    writer.add_summary(summary,global_episodes)

                    writer.flush()
                if global_episodes % save_network_frequency == 0:
                    saver.save(self.sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)
                if local_ep % 128 == 0:
                    self.Network.select_policy(pull=False)

                        
    def train(self, indv_buffer, sess, bootstrap=0.0):
        obs = []
        act = []
        adv = []
        td  = []
        beta= []
        asst = []
        msk = []
        for idx, buffer in enumerate(indv_buffer):
            if len(buffer) == 0:
                alosses.append(None)
                closses.append(None)
                continue
            _history = np.array(buffer)
            observations = _history[:,0]
            actions = _history[:,1]
            rewards = _history[:,2]
            values = _history[:,3]
            behavior_policy = _history[:,4]
            assist_observations = _history[:,5]
            masks = _history[:,6]
            
            rewards_ext = np.append(rewards, [bootstrap[idx]])
            #discounted_rewards = discount_rewards(rewards_ext,gamma)[:-1]
            value_ext = np.append(values, [bootstrap[idx]])
            td_targets = rewards + gamma * value_ext[1:]
            advantages = rewards + gamma * value_ext[1:] - value_ext[:-1]
            advantages = discount_rewards(advantages,gamma)
            
            obs.append(observations)
            act.append(actions)
            adv.append(advantages)
            td.append(td_targets)
            beta.append(behavior_policy)
            asst.append(assist_observations)
            msk.append(masks)
            
        alosses, closs = self.Network.update_full(obs,act,adv,td,beta,asst,msk)
        
        return alosses, closs
    

## Run

In [ ]:
# Global Network
with tf.name_scope('Global_Step'):
    global_step = tf.Variable(0, trainable=False, name='global_step')
    global_step_next = tf.assign_add(global_step, 1)

name = 'Main'
worker = Worker(name, None, sess, global_step=global_step)
print(f'{name} initiated')
saver = tf.train.Saver(max_to_keep=3)
writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")

saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)    
global_episodes = sess.run(global_step)
worker.work(saver, writer)

Main initiated
Initialized Variables
  3560/300000 [..............................] - ETA: 283:53:48